In [ ]:
# Cell 1: Imports & config

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from typing import List, Tuple, Optional

# Match this to the logger (logs/ or tools/)
LOG_DIR = Path("logs")  # or Path("tools")

# Mechanical config (must match firmware)
DEG_PER_STEP = 0.225  # same as in C++

# How many recent logs to analyze
NUM_LOGS_TO_LOAD = 1   # set to 1 for single, >1 to compare

print(f"Log directory: {LOG_DIR.resolve()}")


In [ ]:
# Cell 2: Helpers to find and load logs

from pathlib import Path

def find_log_dir(log_dir: Path) -> Path:
    """Try a few sensible locations for the log directory.

    - If log_dir exists as given, return it.
    - Otherwise search upward from the current working directory and
      return the first parent path containing the requested folder.
    - As a last resort look for a "tools" folder next to the notebook.
    """
    if log_dir.exists():
        return log_dir

    cwd = Path.cwd()
    # Search current working directory and parents (repo-root friendliness)
    for p in [cwd] + list(cwd.parents):
        candidate = p / str(log_dir)
        if candidate.exists():
            print(f"Using log directory: {candidate}")
            return candidate

    # fallback to tools/ (older sessions store logs here)
    candidate = cwd / "tools"
    if candidate.exists():
        print(f"Falling back to: {candidate}")
        return candidate

    # not found
    return log_dir


def list_logs(log_dir: Path) -> List[Path]:
    resolved_dir = find_log_dir(log_dir)
    if not resolved_dir.exists():
        print(f"No logs found. Checked: {log_dir} and parent paths (cwd={Path.cwd()}).")
        return []

    entries = sorted(
        resolved_dir.glob("balance_*.csv"),
        key=lambda f: f.stat().st_mtime,
        reverse=True,
    )

    if not entries:
        print(f"No balance_*.csv files found in {resolved_dir}")

    return entries


def load_log(path: Path) -> pd.DataFrame:
    """Load a single log and add derived columns."""
    df = pd.read_csv(path)
    if df.empty:
        raise ValueError(f"Empty CSV: {path}")

    # Normalize local time
    if "time_s" in df.columns:
        df["t"] = df["time_s"] - df["time_s"].iloc[0]
    else:
        df["t"] = np.arange(len(df)) * 0.008  # fallback: assume dt=8ms

    # Basic sanity: sort by time just in case
    df = df.sort_values("t").reset_index(drop=True)

    # dt between samples
    df["dt"] = df["t"].diff().fillna(0.0)

    # Commanded steps (integral of control_output over time)
    if "control_output" in df.columns:
        df["cmd_steps_inc"] = df["control_output"] * df["dt"]
        df["cmd_steps"] = df["cmd_steps_inc"].cumsum()
        df["cmd_angle_deg"] = df["cmd_steps"] * DEG_PER_STEP
    else:
        df["cmd_steps"] = np.nan
        df["cmd_angle_deg"] = np.nan

    return df

logs = list_logs(LOG_DIR)
if not logs:
    print("No logs found.")
else:
    print("Available logs (newest first):")
    for i, p in enumerate(logs[:10], 1):
        print(f"{i:2d}: {p.name}")


In [ ]:
# Cell 3: Load last N logs

if not logs:
    dfs: List[Tuple[str, pd.DataFrame]] = []
else:
    n = min(NUM_LOGS_TO_LOAD, len(logs))
    dfs = []
    for p in logs[:n]:
        try:
            df = load_log(p)
            dfs.append((p.name, df))
        except Exception as e:
            print(f"Skipping {p.name}: {e}")

if not dfs:
    print("No usable logs loaded.")
else:
    print(f"Loaded {len(dfs)} log(s).")
    # For convenience, first log as df
    name0, df0 = dfs[0]
    print(f"Using {name0} as primary for plotting.")


In [ ]:
# Cell 4: Plot pendulum & base angle vs time for the loaded logs

if "dfs" not in globals() or not dfs:
    print("No data.")
else:
    # Plot each log separately so you can inspect the behavior for each session
    for idx, (name, df) in enumerate(dfs, start=1):
        fig, ax = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

        if "pendulum_deg" in df.columns:
            ax[0].plot(df["t"], df["pendulum_deg"], label="Pendulum angle (sens.)")
            ax[0].axhline(0.0, linestyle="--", alpha=0.3)
            ax[0].set_ylabel("Pendulum (deg)")
            ax[0].legend()
            ax[0].grid(True, alpha=0.3)

        if "base_deg" in df.columns:
            ax[1].plot(df["t"], df["base_deg"], label="Base angle (encoder)")
            ax[1].axhline(0.0, linestyle="--", alpha=0.3)
            ax[1].set_ylabel("Base (deg)")
            ax[1].set_xlabel("Time (s)")
            ax[1].legend()
            ax[1].grid(True, alpha=0.3)

        fig.suptitle(f"Angles vs Time — {name} ({idx}/{len(dfs)})")
        plt.tight_layout()
        plt.show()


In [ ]:
# Cell 5: Commanded vs measured base angle vs time (for each loaded log)

if "dfs" not in globals() or not dfs:
    print("No data.")
else:
    for idx, (name, df) in enumerate(dfs, start=1):
        if "base_deg" not in df.columns or "cmd_angle_deg" not in df.columns:
            print(f"{name}: Missing columns for commanded vs measured comparison.")
            continue

        fig, ax = plt.subplots(figsize=(10, 5))
        ax.plot(df["t"], df["base_deg"], label="Base angle (encoder)")
        ax.plot(df["t"], df["cmd_angle_deg"], label="Commanded stepper angle", linestyle="--")
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Angle (deg)")
        ax.set_title(f"Commanded vs Measured Base Angle – {name} ({idx}/{len(dfs)})")
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
# Cell 6: Control output vs time (microcontroller time vs steps/s) for each log

if "dfs" not in globals() or not dfs:
    print("No data.")
else:
    for idx, (name, df) in enumerate(dfs, start=1):
        if "control_output" not in df.columns:
            print(f"{name}: No control_output column found.")
            continue

        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(df["t"], df["control_output"])
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Steps/sec (Hz)")
        ax.set_title(f"Control Effort – {name} ({idx}/{len(dfs)})")
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
# Cell 7: Phase portrait (pendulum angle vs velocity) for each loaded log

if "dfs" not in globals() or not dfs:
    print("No data.")
else:
    for idx, (name, df) in enumerate(dfs, start=1):
        if "pendulum_deg" not in df.columns or "pendulum_vel" not in df.columns:
            print(f"{name}: Missing pendulum columns.")
            continue

        fig, ax = plt.subplots(figsize=(6, 6))
        ax.plot(df["pendulum_deg"], df["pendulum_vel"], alpha=0.7)
        ax.plot(df["pendulum_deg"].iloc[0], df["pendulum_vel"].iloc[0], "go", label="Start")
        ax.plot(df["pendulum_deg"].iloc[-1], df["pendulum_vel"].iloc[-1], "rx", label="End")
        ax.set_xlabel("Pendulum angle (deg)")
        ax.set_ylabel("Pendulum vel (deg/s)")
        ax.set_title(f"Phase Portrait – {name} ({idx}/{len(dfs)})")
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
# Cell 8: Angle vs step (what you explicitly asked for) for each log

if "dfs" not in globals() or not dfs:
    print("No data.")
else:
    for idx, (name, df) in enumerate(dfs, start=1):
        if "cmd_steps" not in df.columns:
            print(f"{name}: cmd_steps not available.")
            continue

        fig, ax = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

        if "pendulum_deg" in df.columns:
            ax[0].plot(df["cmd_steps"], df["pendulum_deg"])
            ax[0].set_ylabel("Pendulum angle (deg)")
            ax[0].set_title("Pendulum angle vs commanded step count")
            ax[0].grid(True, alpha=0.3)

        if "base_deg" in df.columns:
            ax[1].plot(df["cmd_steps"], df["base_deg"])
            ax[1].set_ylabel("Base angle (deg)")
            ax[1].set_xlabel("Commanded steps (integral of Hz)")
            ax[1].set_title("Base angle vs commanded step count")
            ax[1].grid(True, alpha=0.3)

        fig.suptitle(f"Angles vs Commanded Steps – {name} ({idx}/{len(dfs)})")
        plt.tight_layout()
        plt.show()


In [ ]:
# Cell 9: Summary statistics for all loaded logs

if "dfs" not in globals() or not dfs:
    print("No data.")
else:
    for idx, (name, df) in enumerate(dfs, start=1):
        print("=== SUMMARY ===")
        print(f"File: {name}  ({idx}/{len(dfs)})")
        if "t" in df.columns:
            print(f"Duration: {df['t'].iloc[-1]:.3f} s")
        print(f"Samples: {len(df)}")

        if "pendulum_deg" in df.columns:
            print(f"Pendulum angle RMS: {df['pendulum_deg'].std():.3f} deg")
        if "base_deg" in df.columns:
            print(f"Base angle RMS: {df['base_deg'].std():.3f} deg")
        if "control_output" in df.columns:
            print(f"Control output RMS: {df['control_output'].std():.1f} Hz")

        # Gains & reason
        for col, label in [("kp_balance", "Kp"), ("ki_balance", "Ki"), ("kd_balance", "Kd"), ("kp_motor", "Kp_center")]:
            if col in df.columns and not df[col].isna().all():
                print(f"{label} = {df[col].iloc[0]}")
        if "reason" in df.columns:
            print(f"End reason: {df['reason'].iloc[0]}")
        print("\n")
